# Higuchi Fractal Dimension

In [1]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [2]:
# Load data
data = loadmat('/home/vincent/AAA_projects/MVCS/Neuroscience/downsampled/EEG_DS_Struct_0101.mat')

In [3]:
stim_data = pd.read_excel('/home/vincent/AAA_projects/MVCS/Neuroscience/EEG-tES-Chaos-Neural-Net/Stimulation.xlsx')
stim_data = stim_data.dropna()

stim_data = stim_data.rename(columns={"Unnamed: 13": "Stim Type Block 2",
                                      "Unnamed: 14": "Stim Type Block 3",
                                      "Unnamed: 16": "Stim Amplitude (mA) Block 2",
                                      "Unnamed: 17": "Stim Amplitude (mA) Block 3"})

In [4]:
DSamp = data['DSamp']

triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0] 
fsOld = DSamp[0][0][3][0][0] 
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

In [5]:
print(DSamp, stim_data)

[[(array([[(array([[20.428]]), array([[20429]], dtype=uint16), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64)),
          (array([[619.442]]), array([[619443]], dtype=int32), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64)),
          (array([[619.499]]), array([[619500]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3')),
          (array([[654.746]]), array([[654747]], dtype=int32), array(['0032'], dtype='<U4'), array([[32]], dtype=uint8), array(['Stim Stop'], dtype='<U9'), array([], shape=(1, 0), dtype=float64)),
          (array([[770.515]]), array([[770516]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3')),
          (array([[805.571]])

In [ ]:
import numpy as np

def higuchi_fd(data, kmax):
    """ Compute Higuchi Fractal Dimension of a time series.
    Args:
    data (list or np.array): 1D Time series
    kmax (int): Maximum delay/offset (in number of samples).
    Returns:
    hfd (float): Higuchi Fractal Dimension
    """
    L = []
    x = []
    N = len(data)
    for k in range(1,kmax):
        Lk = []
        for m in range(0,k):
            Lmk = 0
            for i in range(1,int((N-m)/k)):
                Lmk += abs(data[m+i*k] - data[m+i*k-k])
            Lmk = Lmk*(N - 1)/(((N - m)/ k)* k)
            Lk.append(Lmk)
        L.append(np.log(np.mean(Lk)))
        x.append([np.log(1.0/ k), 1])

    (p, r1, r2, s)=np.linalg.lstsq(x, L, rcond=-1)
    return p[0]

DSamp = data['DSamp']
EEGdata = DSamp[0][0][1]

# Transpose the data so each row is a sample and each column is a channel
EEGdata = EEGdata.T

hfd_data = []
kmax = 10  # or other value
for i in range(EEGdata.shape[1]):  # iterate over channels
    channel_data = EEGdata[:, i]   # get data for one channel
    hfd_channel = higuchi_fd(channel_data, kmax)
    hfd_data.append(hfd_channel)

# hfd_data is a list of the Higuchi Fractal Dimension for each channel.